In this task I will have a couple of tasks to performe in my process of sentiment analysis:
## Sentiment Analysis process:
    1.  Analyze the Data and understand how I should handle it.
    2.  Load the necessary data and filter out redundant information.
    3.  Handle the data and prepare it such that it fits the models I will be using. This step will contain a couple of subprocess:
        a. Tokenizing the data.
        b. Sanitizing the data by removing all the stopwords, punctuation and numbers that will only reduce the performance of the Sentiment Analysis model. 
        c. Lemmatizing the data in order to make it more understandable for the model.
        d. Vectorizing in order to fit the data to an graph that I will be using when performing the real Sentiment Analysis. 
    4. Finding the most optimal algorithm.
        a. I will have to try different algorithms with cross validation and use their results in order to find which one fits the data the best. 
        b. In addition I will need to find the most optimal parameters. I will see later how I choose to search for them. 
    5. Training the model with the preprocessed data.
    6. Testing the model with the preprocessed data. 

And then I will be evaluating the data, to see if my score is sufficient or not. 

In [1]:
import json
import nltk
import spacy
import numpy as np
import requests


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# This is for the sentimental analysis more exactly for comparing the different algorithms with eachother. 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

!pip install https://github.com/explosion/spacy-models/releases/download/nb_core_news_sm-3.1.0/nb_core_news_sm-3.1.0.tar.gz

                                              0.0/16.1 MB ? eta -:--:--
                                              0.1/16.1 MB 4.1 MB/s eta 0:00:04
                                              0.1/16.1 MB 1.6 MB/s eta 0:00:11
                                              0.2/16.1 MB 2.1 MB/s eta 0:00:08
                                              0.2/16.1 MB 2.1 MB/s eta 0:00:08
     -                                        0.5/16.1 MB 2.2 MB/s eta 0:00:08
     --                                       0.8/16.1 MB 3.1 MB/s eta 0:00:05
     ---                                      1.3/16.1 MB 4.0 MB/s eta 0:00:04
     ----                                     1.7/16.1 MB 4.8 MB/s eta 0:00:04
     -----                                    2.0/16.1 MB 5.0 MB/s eta 0:00:03
     ------                                   2.5/16.1 MB 5.6 MB/s eta 0:00:03
     --------                                 3.2/16.1 MB 6.5 MB/s eta 0:00:02
     ---------                                4.0/16.1 MB 7

### 1.Analyzing the data & understanding its structure

In [2]:
def data_analysys(json_data) -> list:
    keys_in_data = set()
    print("The size of the data is:", len(json_data))

    for item in json_data:
        keys = item.keys()
        keys_in_data.update(keys)

    print(f'Main keys of this data are: {keys_in_data}')

I will use the function above to analyze the structure of the content of this file. To learn how I should be preprocessing it. And how I can make it more efficient.

In the cell below I simply load the documents from the url's 

In [3]:
# Replace this URL with the raw URL of the file you want to fetch from GitHub
train_url = "https://raw.githubusercontent.com/ltgoslo/norec_sentence/main/binary/train.json"
test_url = "https://raw.githubusercontent.com/ltgoslo/norec_sentence/main/binary/test.json"

# Fetch the content of the file
train_response = requests.get(train_url)
test_response = requests.get(test_url)

# Raise an exception if there was an error fetching the file
train_response.raise_for_status() 
test_response.raise_for_status()

In [4]:
# Get the content of the file as a string
json_training_data = json.loads(train_response.text)
json_test_data = json.loads(test_response.text)
seed = 123

# This is just for analytic reasons. Not for loading the content:
data_analysys(json_training_data)
data_analysys(json_test_data)

The size of the data is: 3894
Main keys of this data are: {'sent_id', 'label', 'text'}
The size of the data is: 583
Main keys of this data are: {'sent_id', 'label', 'text'}


### 2.Loading the data & filtering out what's redundant

As you can see there's an extra, redundant label that I do not have to consider doing my sentiment analysis, and that is the "sent_id" label. The date the message was sent is irrelevant therefore I can remove it from my data to do more efficient model training. 

I will only be using the text and label part of this data so I continue in defining a function that reads only 'label' & 'text' part of the data. 


"data_reading()" extracts the data from the json file provided in a structured way: ["sentance", "sentiment"]. I consider the data not to complex so I decided to not bother with visualizing the data to much.

In [5]:
def data_reading(json_data) -> list:
    # Extract message_data directly using a list comprehension
    message_data = [[category["text"], category["label"]] for category in json_data]

    return message_data

### 3.Handling & Praparing the data for my model.

Before creating the function I was struggling with accessing the data in an efficient way. I wanted to divide: the sanitizing, tokenizing, lemmatizing and vectorizing of the data. But I quickly figured that it would have cost a lot of computation time since I would have to access the same data multiple times in nested for loops. So instead I created preprocessing() which did sanitizing with tokenizing, and then lemmatized the sentances in one go. In addition to that, this function divides the preprocessed data into sentances and sentiments, which are perfectly prepared to be vectorized. 

In [6]:
def preprocessing(data):
    sentences = []
    sentiments = []
    lemmatizer = spacy.load("nb_core_news_sm")  # Norwegian lemmatization model.

    for chunk in data:
        text, sentiment = chunk[0], chunk[1]
        # Here I tokenize the data, and prepare it to run it through a sanitizer.
        pre_lemmatized = lemmatizer(text)


        # Here I add the lemmatized lowercase word to the "lemmatized" list,
        # but only after it passes all the tests, to check if it's a legitimate word.  
        lemmatized = [root_word.lemma_.lower() for root_word in pre_lemmatized
                      if (not root_word.is_punct
                          and not root_word.is_currency
                          and not root_word.is_digit
                          and not root_word.is_space
                          and not root_word.is_stop
                          and not root_word.like_num)]


        # This if statement checks if the list is empty or not.
        # If it is, it simply continues to the next sentance without adding the 
        # sentiment. That provides a reasurance that there will not be any sentiments
        # attached to a empty list.
        if len(lemmatized)==0:
            continue
        else:
            # And here I join each processed word to one sentance in order to do
            # sentance sentimental analysis.
            sentence = ' '.join(lemmatized)
            sentences.append(sentence)
            sentiments.append(sentiment)

    return sentences, sentiments


In the last part of the for loop, where I have that if statement I figured that when filtering out all the stopwords all slang words may be filtered out too. So I needed to add that to not end up with some sentiments being attached to empty lists. 

So I performe the vectorization using CountVectorizer() & LabelEncoder() because that seemed most trivial of every other way that I stumbled upon. I considered if I should use TF-IDF or Bag-of-Words vectorization. But I concluded with TF-IDF seemed to be more fit for bigger datas with variaty of lenghts and inputs. Meanwhile here I have tree prepared documents with the same format which will not vary as much as data from the real world would. 
And my implementation is shown bellow:

In [7]:
def vectornihilation(x_axis, y_axis):
    vectorizer = CountVectorizer()
    labelizer = LabelEncoder()
    X_matrix = vectorizer.fit_transform(x_axis)
    Y_matrix = labelizer.fit_transform(y_axis)

    return X_matrix, Y_matrix, vectorizer, labelizer

And this function gives me a matrix as the x vectors and a numpy array with vector representations of the labels as y.

### 4. Finding the optimal algorithm to performe sentance sentimental analysis.

Now that I have preprocessed data I can use that tougheter with the GridSearchCV() method in order to find the best algorithm and the best parameters for that algorithm to predict the sentiments of the sentances. 

I do that by implementing exhoustive search on the parameters for each of the algorithms.

I choose these tree algorithms because I have read that those are most often tried when performing sentiment analysis. 

In [8]:
def NaiveBayesScore(x_ax, y_ax):
    hypeOmeters = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}

    grid = GridSearchCV(GaussianNB(), hypeOmeters, cv=5)
    grid.fit(X=x_ax, y=y_ax)
    
    optimal_param = grid.best_params_['var_smoothing']
    
    return optimal_param

In [9]:
def LogisticRegressionScore(x_ax, y_ax):
    C_meters = {'C': [0.001, 0.01, 0.1, 1, 10]}

    grid = GridSearchCV(LogisticRegression(), C_meters, cv=5) 
    grid.fit(X=x_ax, y=y_ax)
    
    optimal_param = grid.best_params_['C']

    return optimal_param

In [10]:
def DecissionTreeScore(x_ax, y_ax):
    grido_meters = { 'max_depth': [2, 4, 6, 8, 10], 'min_samples_split': [2, 5, 10, 20] }
    grid = GridSearchCV(DecisionTreeClassifier(), grido_meters, cv=5)

    grid.fit(X=x_ax, y=y_ax)

    optimal_p1, optimal_p2 = grid.best_params_['max_depth'], grid.best_params_['min_samples_split']

    return optimal_p1, optimal_p2


And here I will create a function that will test the different algorithm with their best parameters and present the best algorithm with it's best parameter. 

In [15]:
# Cross-validation:
def OptimalSentimentAnalysis(json_data):
    data = data_reading(json_data) #TODO: Remove the limit.
    sentances, sentiments = preprocessing(data= data)
    x_axis, y_axis, vec, lab = vectornihilation(sentances, sentiments)
    densed_X = x_axis.toarray() # I had to convert the matrix to danse array in order to process it

    Logistic_param = LogisticRegressionScore(x_ax=densed_X, y_ax=y_axis)
    Naive_param = NaiveBayesScore(x_ax=densed_X, y_ax=y_axis)
    DecTree_depth_param, DecTree_sample_split_param = DecissionTreeScore(x_ax=densed_X, y_ax=y_axis)

    Cross_Naive_score = cross_val_score(GaussianNB(var_smoothing=Naive_param), densed_X, y_axis)
    Cross_Logistic_score = cross_val_score(LogisticRegression(C=Logistic_param), densed_X, y_axis)
    Cross_DecTree_score = cross_val_score(DecisionTreeClassifier(max_depth=DecTree_depth_param, min_samples_split=DecTree_sample_split_param), densed_X, y_axis)
    
    print(f'Naive Bayes algorithm scored: {Cross_Naive_score.mean()}')
    print(f'LogisticRegression algorithm scored: {Cross_Logistic_score.mean()}')
    print(f'Decission Tree scored: {Cross_DecTree_score.mean()}')

In [16]:
OptimalSentimentAnalysis(json_training_data)

Naive Bayes algorithm scored: 0.582878949118723
LogisticRegression algorithm scored: 0.7079115397406053
Decission Tree scored: 0.6916704356501497


Here I quickly explain what different parameters does in the different models:
var_smoothing variable chooses how smoothly the variance should be distributed in the Naive Bayes distribution model.

C is a value that regularates the complexity of the Logistic Regregression model. What I try to find is the maximum Likelyhood parameter. Which is the parameter that gives me the optimal values for the likelyhood of getting Y given an X.

max_depth is the maximum depth of the decision tree. A decision tree can become very complex and can overfit the training data if it is allowed to grow too deep. So, setting a maximum depth for the tree can help to prevent overfitting and improve generalization to new data.

min_samples_split chooses how small the decision groups can be, preventing this parameter from being to small can help in preventing overfitting by creating a more general model that fits more data. 



And now I that I have found that Logistic Regression is the model that I am supposed to use I can validate the accuracy of the model before testing it:

In [ ]:
def LogisticSentimentAnalysisRegression_trainANDValidate(training_x, training_y, validating_x, validating_y, module):
    # The evaluation:
    module.fit(training_x, training_y)
    
    validity_responses = module.predict(validating_x)
    
    accuracy = accuracy_score(validating_y, validity_responses)
    report = classification_report(validating_y, validity_responses)

    print(f"{module}:\nAccuracy: {accuracy}\n{report}\n")


In [ ]:
def performance():
    LogiReg = LogisticRegression()  # Instantiating the model.

    train_sentances, train_sentiments = preprocessing(data_reading(json_training_data))

    sentance_vectors_x, sentiment_vectors_y, vectorizer, labelizer = vectornihilation(train_sentances, train_sentiments)
    fixed_SV_x = sentance_vectors_x.toarray()  # Convert the matrix to dense array to process it

    valid_sentances, valid_sentiments = preprocessing(data_reading(json_test_data))
    
    # Use the same vectorizer and labelizer for transforming validation data
    validSentancesVectors_x = vectorizer.transform(valid_sentances)
    validSentimentVectors_Y = labelizer.transform(valid_sentiments)
    
    fixed_ValidSV_x = validSentancesVectors_x.toarray()

    LogisticSentimentAnalysisRegression_trainANDValidate(fixed_SV_x, sentiment_vectors_y, fixed_ValidSV_x, validSentimentVectors_Y, LogiReg)

performance()

0.7152658662092625
